In [1]:
import sys


class TimeoutError(Exception):
    """Timeout exception"""


In [2]:
try:
    raise TimeoutError("Timeout occured")
except TimeoutError as e:
    print(e)

Timeout occured


In [3]:
try:
    raise TimeoutError("Timeout occured")
except:
    ex_type, ex, tb = sys.exc_info()
    print(ex_type, ex, tb)

<class '__main__.TimeoutError'> Timeout occured <traceback object at 0x10543ad40>


In [4]:
try:
    raise TimeoutError("Timeout occured")
except Exception as e:
    print(ex.args, ex.__traceback__)

('Timeout occured',) <traceback object at 0x10543ad40>


In [5]:
class ReadOnlyError(AttributeError):
    """Indicates an attribute is read only"""


In [6]:
try:
    raise ReadOnlyError("Account number is read-only")
except ReadOnlyError as e:
    print(repr(e))

ReadOnlyError('Account number is read-only')


In [7]:
class WebScraperException(Exception):
    """Base exception for WebScrapperTM"""


class HTTPException(WebScraperException):
    """General HTTP exception for WS"""


class InvalidUrlException(HTTPException):
    """Indicates that URL is invalid"""


class TimeoutException(HTTPException):
    """Indicates general timeout exception in HTTP connectivity"""


class PingTimeoutException(TimeoutException):
    """Ping time out"""


class LoadTimeoutException(TimeoutException):
    """Page load time out"""


class ParserException(WebScraperException):
    """General page parsing exception"""


In [8]:
try:
    raise PingTimeoutException("Ping to https://..... timed out")
except WebScraperException as e:
    print(e)

Ping to https://..... timed out


In [9]:
class APIException(Exception):
    """Base API Exception"""


class ApplicationException(APIException):
    """Indicates an application / server error - 5XX HTTP type errors"""


class DBException(ApplicationException):
    """General DB exception"""


class DBConnectionException(DBException):
    """DB connectivity exception"""


class ClientException(APIException):
    """Indicates exception caused by the user - 4XX HTTP type errors"""


class NotFoundException(ClientException):
    """Indicates resource was not found"""


class NotAuthorizedException(ClientException):
    """User is not authorized to perform action on the resource"""


In [10]:
class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type


def lookup_account_by_id(account_id):
    """Mock method to simulate DB connection"""
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f"Account number {account_id} is invalid.")

    if account_id < 100:
        raise DBConnectionException("Failure connecting to the Database.")
    elif account_id < 200:
        raise NotAuthorizedException("User does not have a permissions to read this account.")
    elif account_id < 300:
        raise NotFoundException("Account not found.")
    return Account(account_id, "Savings")

        

In [11]:
from http import HTTPStatus


def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except ApplicationException as e:
        return HTTPStatus.INTERNAL_SERVER_ERROR, str(e)
    except NotFoundException as e:
        return HTTPStatus.NOT_FOUND, f"The account {account_id} does not exist."
    except NotAuthorizedException as e:
        return HTTPStatus.UNAUTHORIZED, "You do not have the proper authorization."
    except ClientException as e:
        return HTTPStatus.BAD_REQUEST, str(e)
    else:
        return HTTPStatus.OK, dict(id=account.account_id, type=account.account_type)

In [12]:
get_account("abc")

(<HTTPStatus.BAD_REQUEST: 400>, 'Account number abc is invalid.')

In [13]:
get_account(50)

(<HTTPStatus.INTERNAL_SERVER_ERROR: 500>,
 'Failure connecting to the Database.')

In [14]:
get_account(150)

(<HTTPStatus.UNAUTHORIZED: 401>, 'You do not have the proper authorization.')

In [15]:
get_account(250)

(<HTTPStatus.NOT_FOUND: 404>, 'The account 250 does not exist.')

In [16]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})

In [17]:
import json
from datetime import datetime, timezone


class APIException(Exception):
    """Base API Exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_error_message = "API exception occcured."
    user_error_message = "We are sorry! An unexpected error occured on our end."

    def __init__(self, *args, user_error_message = None):
        if args:
            self.internal_error_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_error_message)

        if user_error_message is not None:
            self.user_error_message = user_error_message

    def to_json(self):
        err_object = dict(
            status=self.http_status, message=self.user_error_message,
        )
        return json.dumps(err_object)

    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "http_status": self.http_status,
            "message": self.args[0] if self.args else self.internal_error_message,
            "args": self.args[1:],
        }
        print(f"EXCEPTION\n{datetime.now(timezone.utc).isoformat()}: {exception}")


class ApplicationException(APIException):
    """Indicates an application / server error - 5XX HTTP type errors"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_error_message = "Generic application exception occcured."
    user_error_message = "We are sorry! An unexpected error occured on our end."


class DBException(ApplicationException):
    """General DB exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_error_message = "DB exception occcured."
    user_error_message = "We are sorry! An unexpected error occured on our end."


class DBConnectionException(DBException):
    """DB connectivity exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_error_message = "DB connection exception occcured."
    user_error_message = "We are sorry! An unexpected error occured on our end."


class ClientException(APIException):
    """Indicates exception caused by the user - 4XX HTTP type errors"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_error_message = "Client submited a bar request."
    user_error_message = "Bad request was received."


class NotFoundException(ClientException):
    """Indicates resource was not found"""
    http_status = HTTPStatus.NOT_FOUND
    internal_error_message = "Client requested not found resource."
    user_error_message = "Requested object does not exist."


class NotAuthorizedException(ClientException):
    """User is not authorized to perform action on the resource"""
    http_status = HTTPStatus.UNAUTHORIZED
    internal_error_message = "Client not authorized to perform the operation."
    user_error_message = "You are not authorized."

In [18]:

def lookup_account_by_id(account_id):
    """Mock method to simulate DB connection"""
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(
            f"Account number {account_id} is invalid.", 
            f"account_id={account_id}", 
            "type error - account number not an integer",
        )

    if account_id < 100:
        raise DBConnectionException("Failure connecting to the Database.", "db=db01")
    elif account_id < 200:
        raise NotAuthorizedException("User does not have a permissions to read this account.", f"account_id={account_id}")
    elif account_id < 300:
        raise NotFoundException("Account not found.", f"account_id={account_id}")
    return Account(account_id, "Savings")


In [19]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except APIException as e:
        e.log_exception()
        return e.to_json()
    else:
        return HTTPStatus.OK, dict(id=account.account_id, type=account.account_type)

In [20]:
get_account("abc")

EXCEPTION
2024-07-02T09:47:16.584179+00:00: {'type': 'ClientException', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'message': 'Account number abc is invalid.', 'args': ('account_id=abc', 'type error - account number not an integer')}


'{"status": 400, "message": "Bad request was received."}'

In [21]:
get_account(50)

EXCEPTION
2024-07-02T09:47:16.598026+00:00: {'type': 'DBConnectionException', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'Failure connecting to the Database.', 'args': ('db=db01',)}


'{"status": 500, "message": "We are sorry! An unexpected error occured on our end."}'

In [22]:
get_account(150)

EXCEPTION
2024-07-02T09:47:16.617620+00:00: {'type': 'NotAuthorizedException', 'http_status': <HTTPStatus.UNAUTHORIZED: 401>, 'message': 'User does not have a permissions to read this account.', 'args': ('account_id=150',)}


'{"status": 401, "message": "You are not authorized."}'

In [23]:
get_account(250)

EXCEPTION
2024-07-02T09:47:16.640898+00:00: {'type': 'NotFoundException', 'http_status': <HTTPStatus.NOT_FOUND: 404>, 'message': 'Account not found.', 'args': ('account_id=250',)}


'{"status": 404, "message": "Requested object does not exist."}'

In [24]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})